### 文字處理
正、負評價判別


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, utils, layers, models, optimizers, metrics, losses
from tensorflow.keras.preprocessing import sequence

seed = 13
tf.random.set_seed(seed)
np.random.seed(seed)

word_index = datasets.imdb.get_word_index()
index_word = {v:k for k,v in word_index.items()}

In [5]:
top_words = 10000
max_review_length = 80

def data_scale_down(train, test, scale):
    (x_train,y_train),(x_test,y_test) = train, test
    num_train = len(x_train)
    num_test = len(x_test)
    train_size = num_train // scale
    test_size = num_test // scale
    x_train, y_train = x_train[:train_size], y_train[:train_size]
    x_test, y_test = x_test[:test_size], y_test[:test_size]
    return (x_train,y_train), (x_test,y_test)

def imdb_dataset():
    (x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=top_words)
    (x_train,y_train), (x_test,y_test) = data_scale_down((x_train,y_train), (x_test,y_test),10)
    print("x_train.shape",x_train.shape)
    print("y_train.shape",y_train.shape)
    print("x_test.shape",x_test.shape)
    print("y_test.shape",y_test.shape)
    x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
    x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
    
    return (x_train, y_train), (x_test, y_test)
    
(x_train, y_train), (x_test, y_test) = imdb_dataset()

x_train.shape (2500,)
y_train.shape (2500,)
x_test.shape (2500,)
y_test.shape (2500,)
x_train shape: (2500, 80)
x_test shape: (2500, 80)


In [6]:
(x, y), (x_t, y_t) = datasets.imdb.load_data(num_words=top_words)
print("y_train: ", y_train[0])
print("\nx_train:")
print(x_train[0])
print("\nmax_review_length")
print(x[0][-max_review_length:])
print("\norigin data")
print(x[0])
#del x,y,x_t,y_t

y_train:  1

x_train:
[  15  256    4    2    7 3766    5  723   36   71   43  530  476   26
  400  317   46    7    4    2 1029   13  104   88    4  381   15  297
   98   32 2071   56   26  141    6  194 7486   18    4  226   22   21
  134  476   26  480    5  144   30 5535   18   51   36   28  224   92
   25  104    4  226   65   16   38 1334   88   12   16  283    5   16
 4472  113  103   32   15   16 5345   19  178   32]

max_review_length
[15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]

origin data
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 45

In [12]:
print("x_train:")
for i in range(len(x_train[0])):
    print(index_word[x_train[0][i]],end=" ")
print()
print("\norigin data:")
for i in range(len(x[0])):
    print(index_word[x[0][i]],end=" ")

x_train:
for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have two of script their with her nobody most that with wasn't to with armed acting watch an for with heartfelt film want an 

origin data:
the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her

In [19]:
print("diction_size:",len(index_word))
print("example index 1-11")
for i in range(1,11):
    print(index_word[i], end=" ")

diction_size: 88584
example index 1-11
the and a of to is br in it i 

In [50]:
#del RNN
class RNN(models.Model):
    def __init__(self, units, num_classes, num_layers):
        super(RNN, self).__init__()
        self.units = units
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.model = self.create_model()
        
        # self.cells = [keras.layers.LSTMCell(units) for _ in range(num_layers)]
        # self.rnn = keras.layers.RNN(self.cells, unroll=True)
        # self.cells = (keras.layers.LSTMCell(units) for _ in range(num_layers))
        # #
        # self.rnn = keras.layers.RNN(self.cells, return_sequences=True, return_state=True)
        # self.rnn = keras.layers.LSTM(units, unroll=True)
        # self.rnn = keras.layers.StackedRNNCells(self.cells)
        
    def call(self, inputs):
        outputs = self.model(inputs)
        return outputs
    
    def create_model(self):
        model = models.Sequential()
        model.add(layers.Input(shape=(max_review_length,)))
        model.add(layers.Embedding(input_dim=top_words, output_dim=100))# , input_length=max_review_length
        model.add(layers.LSTM(units=self.units, return_sequences=True))
        model.add(layers.LSTM(units=self.units))
        model.add(layers.Dense(units=1)) # , activation="softmax"
        return model


In [51]:
units = 64
num_classes = 2
batch_size = 32
epochs = 1
model = RNN(units, num_classes, num_layers=2)
#model.build(input_shape=(None, max_review_length, ))
model.compile(optimizer=optimizers.Adam(0.05),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [52]:
def train():
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test), verbose=1)
    scores = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)
    print("Final test loss {:.4f} and accuracy {:.4f} :".format(scores[0],scores[1]))
    
train()

Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 44s 18ms/sample - loss: 0.7140 - accuracy: 0.4996 - val_loss: 0.6885 - val_accuracy: 0.5248
2500/1 - 7s - loss: 0.6732 - accuracy: 0.5248
Final test loss 0.6885 and accuracy 0.5248 :


In [53]:
model.summary()

Model: "rnn_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_9 (Sequential)    (None, 1)                 1075329   
Total params: 1,075,329
Trainable params: 1,075,329
Non-trainable params: 0
_________________________________________________________________


In [55]:
predictions = model.predict(x_test[0:10])
print("prdictions:",end=" ")
for i in range(10):
    print("%.4f"%predictions[i], end=" ")
print("\nlabels:\t\t", end=" ")
for i in range(10):
    print(y_train[i], end="\t")

prdictions: -0.0631 -0.0124 0.0404 -0.0170 -0.0184 -0.0141 0.0659 -0.3010 0.0568 0.1267 
labels:		 1	0	0	1	0	0	1	0	1	0	